# Instrumental variables

## IV example on mock dataset

### Constructing the dataset

Create four random series of length $N=1000$

- $x$: education
- $y$: salary
- $z$: ambition
- $q$: early smoking 

such that:

1. $x$ and $z$ cause $y$
2. $z$ causes $x$
3. $q$ is correlated with $x$, not with $z$


A problem arises when the confounding factor $z$ is not observed. In that case, we can estimate the direct effect of $x$ on $y$ by using $q$ as an instrument.

Create a dataset `df`


In [57]:
import numpy as np
import pandas as pd

In [58]:
N = 100000

In [59]:
ϵ_z = np.random.randn(N)*0.1
ϵ_x = np.random.randn(N)*0.1
ϵ_q = np.random.randn(N)*0.01
ϵ_y = np.random.randn(N)*0.01

In [60]:
z = 1 + ϵ_z
x = 0.1 + z + ϵ_x
q = 0.5 + 0.1234*ϵ_x + ϵ_q
y  = 1.0 + 0.9*x + 0.4*z + ϵ_y

In [61]:
df = pd.DataFrame({
    "x": x,
    "y": y,
    "z": z,
    "q": q
})

In [62]:
df.corr()


,x,y,z,q
x,1.000000,0.981870,0.705850,0.552115
y,0.981870,1.000000,0.819707,0.443706
z,0.705850,0.819707,1.000000,-0.000030
q,0.552115,0.443706,-0.000030,1.000000


### Naive approach

Use `linearmodels` to  run a regression estimating the effect of $x$ on $y$ (not the slight API change w.r.t. `statsmodels`). Comment.

In [69]:
from linearmodels import OLS

In [70]:
model = OLS.from_formula("y ~ x", df)
res = model.fit()
res.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                            OLS Estimation Summary                            
==============================================================================
Dep. Variable:                      y   R-squared:                      0.9641
Estimator:                        OLS   Adj. R-squared:                 0.9641
No. Observations:              100000   F-statistic:                 2.687e+06
Date:                Tue, Feb 13 2024   P-value (F-stat)                0.0000
Time:                        19:51:40   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept      1.1803     0.0007     1586.7     0.0000      1.1788      1.1817
x              1.0997     0.0007     1639.2     0.0000      1.0984      1.1010
==============================================================================
"""

> your comment

__Assume briefly that `z` is known and control the regression by `z`. What happens?__

In [71]:
# your code

### Instrumental variable

Use $q$ to instrument the effect of x on y. Comment.

In [74]:
# difference between linearmodels and statsmodels:
# linearmodels does not include the constant by defulat

In [77]:
from linearmodels import IV2SLS
formula = (
    # your formula
)
mod = IV2SLS.from_formula(formula, df)
res = mod.fit()
res

AttributeError: 'tuple' object has no attribute 'strip'

## Return on Education

We follow the excellent R [tutorial](https://www.econometrics-with-r.org/12-6-exercises-10.html) from the (excellent) *Econometrics with R* book.

The goal is to measure the effect of schooling on earnings, while correcting the endogeneity bias by using distance to college as an instrument.

__Download the college distance dataset with `statsmodels`. Describe the dataset and extract the dataframe. Plot an histogram of distance to college.__

https://vincentarelbundock.github.io/Rdatasets/datasets.html

In [84]:
import statsmodels.api as sm
ds = sm.datasets.get_rdataset("CollegeDistance", "AER")

In [85]:
# describe dataset

In [86]:
df = ds.data

In [87]:
# describe dataframe

__Run the naive regression $income=\beta_0 + \beta_1 \text{education} + u$__



In [92]:
from linearmodels import OLS

In [93]:
model = OLS.from_formula("C(income) ~ education", df)
res =model.fit()
res.summary

/home/pablo/.local/opt/micromamba/envs/escp/lib/python3.12/site-packages/linearmodels/shared/linalg.py:64: RuntimeWarning: invalid value encountered in sqrt
  return vecs @ np.diag(1 / np.sqrt(vals)) @ vecs.T


TypeError: only length-1 arrays can be converted to Python scalars

In [94]:
# education variable takes string values ("high" or "low"). 
# we need to convert them into 1 and 0 first
df['income_binary'] = (df['income'] == "high")*1

In [95]:
model = api.ols("income_binary ~ education", df)
result = model.fit()

In [96]:
model = OLS.from_formula("income_binary ~ education", df)
res =model.fit()
res.summary

<class 'linearmodels.compat.statsmodels.Summary'>
"""
                            OLS Estimation Summary                            
==============================================================================
Dep. Variable:          income_binary   R-squared:                      0.0480
Estimator:                        OLS   Adj. R-squared:                 0.0478
No. Observations:                4739   F-statistic:                    227.43
Date:                Tue, Feb 13 2024   P-value (F-stat)                0.0000
Time:                        20:00:23   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
==============================================================================
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
Intercept     -0.4780     0.0499    -9.5702     0.0000     -0.5759     -0.3801
education      0.0555     0.0037     15.081     0.0000      0.0483      0.0627
==============================================================================
"""

In [36]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          income_binary   R-squared:                       0.048
Model:                            OLS   Adj. R-squared:                  0.048
Method:                 Least Squares   F-statistic:                     239.0
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           1.22e-52
Time:                        09:34:48   Log-Likelihood:                -2853.5
No. Observations:                4739   AIC:                             5711.
Df Residuals:                    4737   BIC:                             5724.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.4780      0.050     -9.567      0.000      -0.576      -0.380
education      0.0555      0.004     15.460      0.000       0.048       0.063
==============================================================================
Omnibus:                     1510.859   Durbin-Watson:                   1.791
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              795.036
Skew:                           0.871   Prob(JB):                    2.29e-173
Kurtosis:                       2.003   Cond. No.                         109.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The p-value associated to the Fisher statistics is negligible: the model is globally significant. 
Predictive power is very low (R^2 ~ 5%): the effect of education on income is small w.r.t. to the effect of other factors.

Coefficients for interecept and education terms are significant at the 0.1% threshold.

In [38]:
df.head()

,gender,ethnicity,score,fcollege,mcollege,home,urban,unemp,wage,distance,tuition,education,income,region,income_binary
1,male,other,39.150002,yes,no,yes,yes,6.2,8.09,0.2,0.88915,12,high,other,1
2,female,other,48.869999,no,no,yes,yes,6.2,8.09,0.2,0.88915,12,low,other,0
3,male,other,48.740002,no,no,yes,yes,6.2,8.09,0.2,0.88915,12,low,other,0
4,male,afam,40.400002,no,no,yes,yes,6.2,8.09,0.2,0.88915,12,low,other,0
5,female,other,40.480000,no,no,no,yes,5.6,8.09,0.4,0.88915,13,low,other,0


In [40]:
df['ethnicity'].unique()

array(['other', 'afam', 'hispanic'], dtype=object)

__Augment the regression with `unemp`, `hispanic`, `af-am`, `female` and `urban`__

In [46]:
from patsy import Treatment

In [54]:
model = api.ols("income_binary ~ education + C(gender,Treatment(reference='male')) + C(ethnicity,Treatment(reference='other')) + urban + unemp", df)
result = model.fit()

In [55]:
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          income_binary   R-squared:                       0.083
Model:                            OLS   Adj. R-squared:                  0.082
Method:                 Least Squares   F-statistic:                     71.34
Date:                Tue, 15 Mar 2022   Prob (F-statistic):           2.00e-85
Time:                        09:51:02   Log-Likelihood:                -2764.9
No. Observations:                4739   AIC:                             5544.
Df Residuals:                    4732   BIC:                             5589.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==========================================================================================================================
                                                             coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                 -0.2433      0.054     -4.528      0.000      -0.349      -0.138
C(gender, Treatment(reference='male'))[T.female]          -0.0490      0.013     -3.861      0.000      -0.074      -0.024
C(ethnicity, Treatment(reference='other'))[T.afam]        -0.1235      0.018     -6.920      0.000      -0.159      -0.089
C(ethnicity, Treatment(reference='other'))[T.hispanic]    -0.1532      0.017     -9.197      0.000      -0.186      -0.121
urban[T.yes]                                              -0.0470      0.015     -3.073      0.002      -0.077      -0.017
education                                                  0.0511      0.004     14.422      0.000       0.044       0.058
unemp                                                     -0.0115      0.002     -5.006      0.000      -0.016      -0.007
==============================================================================
Omnibus:                     1212.294   Durbin-Watson:                   1.836
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              697.989
Skew:                           0.813   Prob(JB):                    2.71e-152
Kurtosis:                       2.055   Cond. No.                         136.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

__Comment the results and explain the selection problem__

All coefficients are significant at the 1% level. 
Prediction power is higher : R^2 about 8%.

__Explain why distance to college might be used to instrument the effect of schooling.__

We need an instrument that:

- is correlated with schooling: 
    - distance to college affects chances to go to university hence schooling
- independent from other factors (gender, ethnicity, ...)
    
The effect of "distance to college" on income, is only through its effect on education.

__Run an IV regression, where `distance` is used to instrument schooling.__

look at: 
    https://bashtage.github.io/linearmodels/
   (two-stage least squares)

In [ ]:
# remember that linearmodels does not include constants by default
# we take the same formula and add the constant

In [62]:
from linearmodels import IV2SLS
formula = (
"income_binary ~ 1 + [education~distance] + C(gender,Treatment(reference='male')) + C(ethnicity,Treatment(reference='other')) + urban + unemp"
)
mod = IV2SLS.from_formula(formula, df)
res = mod.fit()
res

IV-2SLS Estimation Summary                          
==============================================================================
Dep. Variable:          income_binary   R-squared:                     -0.2734
Estimator:                    IV-2SLS   Adj. R-squared:                -0.2750
No. Observations:                4739   F-statistic:                    213.68
Date:                Tue, Mar 15 2022   P-value (F-stat)                0.0000
Time:                        10:12:51   Distribution:                  chi2(6)
Cov. Estimator:                robust                                         
                                                                              
                                                   Parameter Estimates                                                    
==========================================================================================================================
                                                        Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
--------------------------------------------------------------------------------------------------------------------------
Intercept                                                 -2.3763     0.5316    -4.4699     0.0000     -3.4182     -1.3343
C(gender, Treatment(reference='male'))[T.female]          -0.0456     0.0150    -3.0301     0.0024     -0.0750     -0.0161
C(ethnicity, Treatment(reference='other'))[T.afam]        -0.0456     0.0283    -1.6123     0.1069     -0.1011      0.0098
C(ethnicity, Treatment(reference='other'))[T.hispanic]    -0.1075     0.0223    -4.8322     0.0000     -0.1511     -0.0639
urban[T.yes]                                              -0.0527     0.0182    -2.8947     0.0038     -0.0884     -0.0170
unemp                                                     -0.0101     0.0027    -3.7771     0.0002     -0.0153     -0.0048
education                                                  0.2032     0.0378     5.3800     0.0000      0.1292      0.2773
==========================================================================================================================

Endogenous: education
Instruments: distance
Robust Covariance (Heteroskedastic)
Debiased: False
IVResults, id: 0x7f3f2323f0d0

__Comment the results. Compare with the R tutorials.__

R^2 is negative, but we can't compare it with the non-IV regression.

All coefficients are significant at the 1% level, save for ethnicity (for category "afam").
With the instrumentation strategy, the effect of education on salary, is 4 times higher than without it.